## CHARGEMENT

In [21]:
import pandas as pd

Le fichier est chargé :

In [27]:
data = pd.read_csv("C:\\Users\\olake\\Desktop\\PROJETS_CAS_ENTREPRISE\\PROJET_1\\data\\donnees_brutes\\Book_Reviews.csv")
data.head()

,Unnamed: 0,Book,Review,Review Date
0,0,To Kill a Mockingbird,/// gentle reminder that this is not the time ...,"March 24, 2022"
1,1,To Kill a Mockingbird,\r\n|\r\n|6.0 stars. I know I am risking a ser...,"May 24, 2011"
2,2,To Kill a Mockingbird,\r\n|\r\n|Looking for a new book but don't wan...,"December 10, 2020"
3,3,To Kill a Mockingbird,"To Kill a Mockingbird, Harper Lee|To Kill a Mo...","July 1, 2022"
4,4,To Kill a Mockingbird,Why is it when I pick up | To Kill A Mockingbi...,"October 25, 2009"


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32081 entries, 0 to 32080
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   32081 non-null  int64 
 1   Book         32081 non-null  object
 2   Review       31772 non-null  object
 3   Review Date  32081 non-null  object
dtypes: int64(1), object(3)
memory usage: 1002.7+ KB


## NETTOYAGE

Nettoyage des titres :

In [30]:
import re # utilisé pour manipuler du texte avec des expressions 
            # régulières (supprimer des caractères, nettoyer les 
            # espaces, filtrer les lettres et chiffres)
import unicodedata # pour gérer les accents

def nettoyer_texte(text):
    # Mettre tous les mots du texte en minuscule afin d'uniformiser le texte
    if pd.isna(text):
        return ""
    text = str(text).lower()
    
    # Supprimer les accents
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )
    
    # Forcer un encodage propre (UTF-8) pour éliminer les caractères invisibles ou corrompus,
    # éviter des erreurs durant l'analyse 
    text = text.encode('utf-8', 'ignore').decode('utf-8')

    # Supprimer les caractères spéciaux
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    # Supprimer les espaces nettoyés
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [31]:
data["avis_nettoy"] = data["Review"].apply(nettoyer_texte)

data[["Review", "avis_nettoy"]].head()

,Review,avis_nettoy
0,/// gentle reminder that this is not the time ...,gentle reminder that this is not the time to r...
1,\r\n|\r\n|6.0 stars. I know I am risking a ser...,60 stars i know i am risking a serious film at...
2,\r\n|\r\n|Looking for a new book but don't wan...,looking for a new book but dont want to commit...
3,"To Kill a Mockingbird, Harper Lee|To Kill a Mo...",to kill a mockingbird harper leeto kill a mock...
4,Why is it when I pick up | To Kill A Mockingbi...,why is it when i pick up to kill a mockingbird...


In [35]:
from langdetect import detect

def avis_en_anglais(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Filtrer uniquement les avis en anglais
data = data[data["Review"].apply(avis_en_anglais)]


In [36]:
data[data["avis_nettoy"].str.len()==0] # je checke s'il y a des titres vides après ce nettoyage

,Unnamed: 0,Book,Review,Review Date,avis_nettoy


In [37]:
data = data.dropna(subset=["Review"])

In [39]:
data["avis_nettoy"].str.split().str.len().describe()

count    23051.000000
mean       370.898616
std        404.148650
min          1.000000
25%        112.000000
50%        249.000000
75%        483.000000
max       3652.000000
Name: avis_nettoy, dtype: float64

Suppression des stopwords qui n'apportent rien à l'analyse (the, in, and, ...)

In [40]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def eliminer_stopwords(text):
    mots = text.split()
    mots = [i for i in mots if i not in ENGLISH_STOP_WORDS]
    return " ".join(mots)

data["avis_nettoy"] = data["avis_nettoy"].apply(eliminer_stopwords)

Lemmatisation : ramener chaque mot à sa forme de base (lemme)

In [41]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    mots = text.split()
    lemmes = [lemmatizer.lemmatize(mot) for mot in mots]
    return ' '.join(lemmes)

data["avis_nettoy"] = data["avis_nettoy"].apply(lemmatize_text) 

In [42]:
data.head()

,Unnamed: 0,Book,Review,Review Date,avis_nettoy
0,0,To Kill a Mockingbird,/// gentle reminder that this is not the time ...,"March 24, 2022",gentle reminder time read book reread 2017 don...
1,1,To Kill a Mockingbird,\r\n|\r\n|6.0 stars. I know I am risking a ser...,"May 24, 2011",60 star know risking film 11 moment club upsid...
2,2,To Kill a Mockingbird,\r\n|\r\n|Looking for a new book but don't wan...,"December 10, 2020",looking new book dont want commit check latest...
4,4,To Kill a Mockingbird,Why is it when I pick up | To Kill A Mockingbi...,"October 25, 2009",pick kill mockingbird instantly visited sensor...
5,5,To Kill a Mockingbird,I had a much longer review written for this bo...,"December 17, 2020",longer review written book comment sadly annoy...


Le dataframe nettoyé est sauvegardé en fichier CSV :

In [43]:
data.to_csv("C:\\Users\\olake\\Desktop\\PROJETS_CAS_ENTREPRISE\\PROJET_1\\data\\donnees_nettoy\\livres_final.csv", index=False)